# Notebook to make baseline estimations, in the US

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm
from scipy import stats

%matplotlib inline
plt.style.use('ggplot')

## Helper functions

In [ ]:
def cross_validate(inputs, labels, n, features, output_transforms=None):
    fit_rsquareds = []
    RMSEs = []
    MAEs = []
    SCorrs = []
    pVals = []
    print("\nResults ({} features) :".format(len(features)))
    for i in range(n):
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2)
        X_train1 = X_train[features].copy()
        X_test1 = X_test[features].copy()     
        # Fitting the model
        X2 = sm.add_constant(X_train1)
        est = sm.OLS(y_train, X2)
        est2 = est.fit()
        fit_rsquareds.append(est2.rsquared_adj)
        # Making predictions
        X2 = sm.add_constant(X_test1)
        y_pred = est2.predict(X2)
        # Re-transforming the outputs
        if (output_transforms != None):
            y_pred = output_transforms(y_pred)
            y_test = output_transforms(y_test)
        # Storing the results
        RMSEs.append((mean_squared_error(y_test, y_pred, squared=False)))
        MAEs.append((mean_absolute_error(y_test, y_pred)))
        SCorrs.append(stats.spearmanr(y_test.to_numpy().reshape(y_test.shape[0],), y_pred.to_numpy())[0])
        pVals.append(stats.spearmanr(y_test.to_numpy().reshape(y_test.shape[0],), y_pred.to_numpy())[1])
    # Storing and printing the results
    df = pd.DataFrame()
    df['Adjusted R2'] = fit_rsquareds
    df['RMSE'] = RMSEs
    df['MAE'] = MAEs
    df['Spearman Correlation'] = SCorrs
    print()
    print("Mean Adjusted R2 when fitting : {}".format(np.mean((fit_rsquareds))))
    print()
    print("Mean RMSE : {}".format(np.mean((RMSEs))))
    print("Mean MAE : {}".format(np.mean(MAEs)))
    print("Mean Spearman Correlation : {}".format(np.mean(SCorrs)))
    print("Mean P-Value : {}".format(np.mean(pVals)))
    
    return df

## General pre-processing

### Loading the data

In [ ]:
metrics = pd.read_csv("data/us_metrics_baseline.csv")

states = ["california","illinois"]
frames = []
for i in states:
    new_frame = pd.read_csv("data/indices_per_tract_" + i + ".csv")
    frames.append(new_frame)
indices_per_tract = pd.concat(frames).reset_index(drop=True)

### Merging the data per ward, and filtering wards

In [ ]:
metrics_indices = metrics.merge(indices_per_tract, on="tract").drop(columns=['tract'])

In [ ]:
rows = []
for i in range(metrics_indices.shape[0]):
    if (metrics_indices['count'][i] < 5):
        rows.append(i)
metrics_indices = metrics_indices.drop(rows).reset_index()

## Estimating Education

### Taking the input and output data

In [ ]:
X = metrics_indices.drop(['poverty','unemployment','education','income'],axis=1)
y = metrics_indices[['education']]

### Transforming the data

In [ ]:
X_transformed = X.copy()
X_transformed['count'] = np.log(X_transformed['count'])

y_transformed = y.copy()
y_transformed = np.sqrt(y_transformed)

### Scaling the data

In [ ]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_transformed), columns=['count'])

### Making estimations

In [ ]:
X_ = X_scaled.copy()
y_ = y_transformed.copy()

In [ ]:
results = cross_validate(X_, y_, 100, ['count'], np.square)

### Saving the results to .csv

In [ ]:
results.to_csv("../data/temp_results/us_baseline.csv", index=False)

## Estimating Income

### Taking the input and output data

In [ ]:
X = metrics_indices.drop(['poverty','unemployment','education','income'],axis=1)
y = metrics_indices[['income']]

### Filtering income output
 - Remove the rows with missing income
 - Set the top income to 250,000

In [ ]:
rows_to_remove = []
for i in range(y.shape[0]):
    if (y['income'][i] == "-"):
        rows_to_remove.append(i)
    elif (y['income'][i] == "250,000+"):
        y['income'][i] = 250000
X = X.drop(rows_to_remove).reset_index(drop=True)
y = y.drop(rows_to_remove).reset_index(drop=True)
y['income'] = y['income'].astype(float)

### Transforming the data

In [ ]:
X_transformed = X.copy()
X_transformed['count'] = np.log(X_transformed['count'])

y_transformed = y.copy()
y_transformed = np.sqrt(y_transformed)

### Scaling the data

In [ ]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_transformed), columns=['count'])

### Making estimations

In [ ]:
X_ = X_scaled.copy()
y_ = y_transformed.copy()

In [ ]:
results = cross_validate(X_, y_, 100, ['count'], np.square)

### Saving the results to .csv

In [ ]:
results.to_csv("../data/temp_results/us_baseline_income.csv", index=False)